In [9]:
import torch

# Check if GPU is available
print("CUDA available:", torch.cuda.is_available())

# Check which device is being used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Optional: print GPU name
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

CUDA available: True
Using device: cuda
GPU name: Tesla T4


### Step 1: Install necesscary packages

In [10]:
!pip install matplotlib
!pip install torch numpy transformers datasets tiktoken wandb tqdm

### Step 2: Package imports and configuration

In [11]:
import sys
import os
sys.path.append(os.path.abspath(".."))
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import pickle
from model import GPT, GPTConfig
import random
from tqdm import tqdm
import time
import json
import matplotlib.pyplot as plt
# Configuration
beta = 0.5
device = 'cuda' if torch.cuda.is_available() else 'cpu'
base_lr = 1e-4 #default is 1e-4
epochs = 5
batch_size = 64
max_length =64
num_samples = 1
max_new_tokens = 200
temperature = 0.8
top_k = 200
# tokenizer
with open("meta.pkl", "rb") as f:
    meta = pickle.load(f)
stoi, itos = meta["stoi"], meta["itos"]
def encode(s): return [stoi[c] for c in s]
def decode(l): return ''.join([itos[i] for i in l])

# this the default

### Step 3: Define helper functions

In [12]:
def compute_logprob(input_ids):
    inputs = input_ids[:, :-1]
    targets = input_ids[:, 1:]
    logits, _ = gpt(inputs, full_seq=True)
    B, T, V = logits.size()
    logits_flat = logits.reshape(-1, V)
    targets_flat = targets.reshape(-1)
    loss = F.cross_entropy(logits_flat, targets_flat, ignore_index=0, reduction='none')
    loss = loss.reshape(B, T)
    attention_mask = (targets != 0).float()
    loss = (loss * attention_mask).sum(dim=1) / attention_mask.sum(dim=1)
    return -loss

def pad_or_truncate(seq, max_length):
    return seq[-max_length:] if len(seq) > max_length else seq + [0] * (max_length - len(seq))

def get_batches(lines, batch_size):
    random.shuffle(lines)
    #for l in lines:
    #    print(l[1])
    for i in range(0, len(lines), batch_size):
        batch = lines[i:i+batch_size]
        if len(batch) < batch_size:
            continue
        neg_inputs = [pad_or_truncate(encode(p['negative'] + '\n\n\n\n'), max_length) for p in batch]
        pos_inputs = [pad_or_truncate(encode(p['positive'] + '\n\n\n\n'), max_length) for p in batch]
        neg_tensor = torch.tensor(neg_inputs, dtype=torch.long, device=device)
        pos_tensor = torch.tensor(pos_inputs, dtype=torch.long, device=device)
        yield neg_tensor, pos_tensor

### Step 4: Load the pretrained NanoGPT model

In [13]:
ckpt = torch.load("gpt.pt", map_location=device)
gptconf = GPTConfig(**ckpt['model_args'])
gpt = GPT(gptconf)
state_dict = ckpt['model']
unwanted_prefix = '_orig_mod.'
for k in list(state_dict.keys()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
gpt.load_state_dict(state_dict)
gpt.to(device).train()

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(74, 348)
    (wpe): Embedding(256, 348)
    (drop): Dropout(p=0.2, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=348, out_features=1044, bias=False)
          (c_proj): Linear(in_features=348, out_features=348, bias=False)
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=348, out_features=1392, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=1392, out_features=348, bias=False)
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=348, out_features=74, bias=False)
)

### Step 5: Load Data (**students are required to complete this part!**)

In [14]:
# Load data from pos_neg_pairs.json
import json
import tiktoken
# Loading the json file, CHANGE ADDRESS IF NEEDED
lines = ""
with open("pos_neg_pairs.json", "r", encoding = "utf-8") as f:
 lines = json.load(f)
 print(f"Loaded {len(lines)} pairs.")

Loaded 100000 pairs.


Step 6: Optimizer and Scheduler

In [15]:
grad_clip = 1.0  # Gradient clipping to prevent explosion
anchor_weight_start = 0.2
anchor_weight_end = 0.05
neg_anchor_weight = 0.005

total_steps = len(lines) // batch_size

In [16]:
# AdamW optimizer
import math
optimizer = gpt.configure_optimizers(0.1, base_lr, (0.9, 0.95), device)

# Warmup + Cosine Annealing Scheduler
def warmup_cosine_scheduler(total_steps, base_lr, current_step):
    max_lr = base_lr
    min_lr = 0.1 * base_lr  # Minimum learning rate (scaled down)
    warmup_steps = round(0.03 * total_steps)  # Define warmup duration (first 3% of total steps)

    if current_step <= warmup_steps:
        # Linear warmup phase: gradually increase learning rate from 0 to base_lr
        lr = max_lr * (current_step / warmup_steps)
    elif current_step >= total_steps:
        # After training is complete, set learning rate to min_lr
        lr = min_lr
    else:
        # Cosine annealing phase: decay learning rate smoothly
        p = (current_step - warmup_steps) / (total_steps - warmup_steps)
        lr = min_lr + 0.5 * (max_lr - min_lr) * (1 + math.cos(math.pi * p))

    return lr

# Apply the custom scheduler to the optimizer
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda step: warmup_cosine_scheduler(total_steps * epochs, base_lr, step))


num decayed parameter tensors: 26, with 8,834,328 parameters
num non-decayed parameter tensors: 13, with 4,524 parameters
using fused AdamW: True


### Step 7: Begin training (**students are required to complete this part!**)

In [ ]:
for epoch in range(epochs):
    pbar = tqdm(get_batches(lines, batch_size))
    epoch_loss = 0.0  # Initialize the epoch loss for each epoch
    for step, (neg_tensor, pos_tensor) in enumerate(pbar):
        ##########################################################################
        # Forward pass for negative and positive tensors
        neg_logprob = compute_logprob(neg_tensor)
        pos_logprob = compute_logprob(pos_tensor)

        # Compute the preference term (contrastive loss)
        logit_diff = (pos_logprob - neg_logprob) / beta
        preference_term = -torch.nn.functional.logsigmoid(logit_diff).mean()

        # Adaptive anchor term (adjust the importance of positive/negative samples)
        progress = epoch * total_steps + step  # Track progress over the entire training
        anchor_weight = anchor_weight_start * (1 - progress / (total_steps * epochs)) + anchor_weight_end * (progress / (total_steps * epochs))

        # Dual anchoring: encourage good positives, discourage negatives
        pos_anchor = -anchor_weight * pos_logprob.mean()
        neg_anchor = neg_anchor_weight * neg_logprob.mean()
        anchor_term = pos_anchor + neg_anchor

        # Total loss (contrastive + anchor regularization)
        loss = preference_term + anchor_term

        # Backward pass
        optimizer.zero_grad()  # Zero the gradients before backpropagation
        loss.backward()

        # Clip gradients for stability
        torch.nn.utils.clip_grad_norm_(gpt.parameters(), grad_clip)

        # Optimizer step: update model parameters
        optimizer.step()

        # Track loss for the epoch
        epoch_loss += loss.item()

        # Update progress bar description with loss and learning rate info
        lr = scheduler.get_last_lr()[0]  # Get the current learning rate from scheduler
        pbar.set_description(f"Epoch {epoch + 1}/{epochs} | Step {step}/{total_steps} | Loss {loss.item():.4f} | LR {lr:.2e}")

    # Update the learning rate scheduler at the end of each epoch
    scheduler.step()

    # Print average loss for the epoch
    print(f"Epoch {epoch + 1} completed. Avg Loss: {epoch_loss / total_steps:.4f}")
        ##########################################################################

    ckpt_path = f"./dpo.pt"
    torch.save({
            "model_state_dict": gpt.state_dict(),
            "model_args": ckpt['model_args'],
        }, ckpt_path)
    print(f"Saved checkpoint to {ckpt_path}")


Epoch 1/5 | Step 39/1562 | Loss 10.3540 | LR 0.00e+00: : 40it [00:06,  6.65it/s]

### Step 8: Begin testing (**students are required to complete this part!**)

In [ ]:
# Load the fine-tuned model
ckpt_path = "dpo.pt"
checkpoint = torch.load(ckpt_path, map_location=device)
gptconf = GPTConfig(**checkpoint['model_args'])
gpt = GPT(gptconf).cuda()
try:
    state_dict = checkpoint['model']
except:
    state_dict = checkpoint['model_state_dict']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
gpt.load_state_dict(state_dict)
# Test
gpt.eval()
test_set = ["17+19=?", "3*17=?", "72/4=?", "72-x=34,x=?", "x*11=44,x=?", "3*17=?", "72/4=?", "72-x=34,x=?"]
with torch.no_grad():
    for prompt in test_set:
        prompt_ids = encode(prompt)
        ###########################################################
        prompts_ids_formatted = torch.tensor([prompt_ids],dtype=torch.long,device=device)
        result = gpt.generate(prompts_ids_formatted, max_new_tokens, temperature, top_k)
        decoded_result = decode(result[0].detach().cpu().view(-1).tolist())
        print(decoded_result)
        ###########################################################
